In [1]:
import os
import re

from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, current_timestamp

from delta.tables import DeltaTable

conf = (
    SparkConf()
    .setAppName("Spark minIO Test")
    .set("spark.hadoop.fs.s3a.endpoint", "http://192.168.86.192:9000")
    .set("spark.hadoop.fs.s3a.access.key", os.getenv('MINIO_ROOT_USER'))
    .set("spark.hadoop.fs.s3a.secret.key", os.getenv('MINIO_ROOT_PASSWORD'))
    .set("spark.hadoop.fs.s3a.path.style.access", True)
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .set("spark.driver.memory", "8g")
    .set("spark.executor.memory", "8g")
    .set("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") 
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
)
sc = SparkContext(conf=conf).getOrCreate()
spark = SparkSession(sc).builder.getOrCreate()

23/02/13 20:45:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/13 20:45:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/13 20:45:28 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/02/13 20:45:28 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [2]:
weather_stations_file = sc.textFile('s3a://landing-knmi/weather_stations/*')

In [3]:
weather_stations_file.take(10)

23/02/13 20:45:29 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


['# Opmerking: door stationsverplaatsingen en veranderingen in waarneemmethodieken zijn deze tijdreeksen van uurwaarden mogelijk inhomogeen! Dat betekent dat deze reeks van gemeten waarden niet geschikt is voor trendanalyse. Voor studies naar klimaatverandering verwijzen we naar de gehomogeniseerde dagreeksen <http://www.knmi.nl/nederland-nu/klimatologie/daggegevens> of de Centraal Nederland Temperatuur <http://www.knmi.nl/kennis-en-datacentrum/achtergrond/centraal-nederland-temperatuur-cnt>.',
 '# ',
 '# SOURCE: ROYAL NETHERLANDS METEOROLOGICAL INSTITUTE (KNMI)',
 '# Comment: These time series are inhomogeneous because of station relocations and changes in observation techniques. As a result these series are not suitable for trend analysis. For climate change studies we refer to the homogenized series of daily data <http://www.knmi.nl/nederland-nu/klimatologie/daggegevens> or the Central Netherlands Temperature <http://www.knmi.nl/kennis-en-datacentrum/achtergrond/centraal-nederland-t

In [4]:
weather_stations = (weather_stations_file
                        .filter(lambda x: x.startswith('# STN         ') or re.match('# \d\d\d', x))
                        .map(lambda x: x.replace('# ', ''))
                        .map(lambda x: re.sub('\s\s+', ';', x.strip()))
                        .map(lambda x: x.split(';'))
                   )

In [5]:
weather_stations.take(5)

[['STN', 'LON(east)', 'LAT(north)', 'ALT(m)', 'NAME'],
 ['209', '4.518', '52.465', '0.00', 'IJmond'],
 ['210', '4.430', '52.171', '-0.20', 'Valkenburg Zh'],
 ['215', '4.437', '52.141', '-1.10', 'Voorschoten'],
 ['225', '4.555', '52.463', '4.40', 'IJmuiden']]

In [6]:
header = weather_stations.first()
header

['STN', 'LON(east)', 'LAT(north)', 'ALT(m)', 'NAME']

In [7]:
data = weather_stations.filter(lambda row : row != header)

In [8]:
df = data.toDF(header)
df.show(5)

+---+---------+----------+------+-------------+
|STN|LON(east)|LAT(north)|ALT(m)|         NAME|
+---+---------+----------+------+-------------+
|209|    4.518|    52.465|  0.00|       IJmond|
|210|    4.430|    52.171| -0.20|Valkenburg Zh|
|215|    4.437|    52.141| -1.10|  Voorschoten|
|225|    4.555|    52.463|  4.40|     IJmuiden|
|235|    4.781|    52.928|  1.20|      De Kooy|
+---+---------+----------+------+-------------+
only showing top 5 rows



In [9]:
weather_stations_df = df.select(col('STN').alias('weather_station_code')
                               ,col('LON(east)').alias('longitude')
                               ,col('LAT(north)').alias('latitude')
                               ,col('ALT(m)').alias('altitude')
                               ,col('NAME').alias('weather_station'))
weather_stations_df.show(5)

+--------------------+---------+--------+--------+---------------+
|weather_station_code|longitude|latitude|altitude|weather_station|
+--------------------+---------+--------+--------+---------------+
|                 209|    4.518|  52.465|    0.00|         IJmond|
|                 210|    4.430|  52.171|   -0.20|  Valkenburg Zh|
|                 215|    4.437|  52.141|   -1.10|    Voorschoten|
|                 225|    4.555|  52.463|    4.40|       IJmuiden|
|                 235|    4.781|  52.928|    1.20|        De Kooy|
+--------------------+---------+--------+--------+---------------+
only showing top 5 rows



In [10]:
weather_stations_df = weather_stations_df.withColumn('load_datetime', current_timestamp())

In [11]:
weather_stations_df.show(5)

+--------------------+---------+--------+--------+---------------+--------------------+
|weather_station_code|longitude|latitude|altitude|weather_station|       load_datetime|
+--------------------+---------+--------+--------+---------------+--------------------+
|                 209|    4.518|  52.465|    0.00|         IJmond|2023-02-13 20:46:...|
|                 210|    4.430|  52.171|   -0.20|  Valkenburg Zh|2023-02-13 20:46:...|
|                 215|    4.437|  52.141|   -1.10|    Voorschoten|2023-02-13 20:46:...|
|                 225|    4.555|  52.463|    4.40|       IJmuiden|2023-02-13 20:46:...|
|                 235|    4.781|  52.928|    1.20|        De Kooy|2023-02-13 20:46:...|
+--------------------+---------+--------+--------+---------------+--------------------+
only showing top 5 rows



In [12]:
if not DeltaTable.isDeltaTable(spark, 's3a://bronze-knmi/weather_stations'):
    print("Not a delta table, write the full df")
    weather_stations_df.dropDuplicates().coalesce(1).write.format("delta").mode("overwrite").option("mergeSchema", "true").save('s3a://bronze-knmi/weather_stations')
else:
    weather_stations_df.createOrReplaceTempView('weather_stations')
    spark.sql("""
        create table if not exists bronze_knmi_weather_stations
        using delta 
        location 's3a://bronze-knmi/weather_stations'
        """)
    # We don't want to update values, only insert with new load_date_time when data is changed, so we don't use a when matched
    spark.sql("""
    MERGE INTO bronze_knmi_weather_stations
    USING weather_stations
        ON bronze_knmi_weather_stations.weather_station_code = weather_stations.weather_station_code
        AND bronze_knmi_weather_stations.longitude = weather_stations.longitude
        AND bronze_knmi_weather_stations.latitude = weather_stations.latitude
        AND bronze_knmi_weather_stations.altitude = weather_stations.altitude
        AND bronze_knmi_weather_stations.weather_station = weather_stations.weather_station
    WHEN NOT MATCHED
      THEN INSERT *
    """)
    

ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8ANTLR Tool version 4.7 used for code generation does not match the current runtime version 4.8